In [32]:
import os
import warnings
import importlib
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix, f1_score, accuracy_score, \
    precision_score, recall_score
import xgboost as xgb
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix, f1_score
from sklearn.model_selection import KFold, LeavePGroupsOut
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot as plt
import matplotlib
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from tabpfn import TabPFNClassifier
from sklearn.preprocessing import label_binarize
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.utils.class_weight import compute_sample_weight

%matplotlib inline
# matplotlib.use('TkAgg')
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler  # install via: pip install imbalanced-learn

warnings.filterwarnings('ignore')
import torch
from scipy.stats import skew
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier


from sktime.transformations.panel.rocket import Rocket


seed = 69
torch.manual_seed(seed)
np.random.seed(seed)

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 3080 Ti Laptop GPU


In [33]:
root = '/home/ali/PycharmProjects/tihm/dataset'

dataset_06h = pd.read_csv(os.path.join(root, 'data-06h.csv'))
dataset_12h = pd.read_csv(os.path.join(root, 'data-12h.csv'))
dataset_24h = pd.read_csv(os.path.join(root, 'data-24h.csv'))

In [34]:
# display(dataset_06h)
# display(dataset_12h)
# display(dataset_24h)

In [35]:
# print(dataset_06h['agitation'].equals(dataset_06h['agitation-next']))
# print(np.sum(dataset_06h['agitation']))
# print(np.sum(dataset_06h['agitation-next']))

# print(np.where(dataset_06h['agitation'] == 1))
# print(np.where(dataset_06h['agitation-next'] == 1))

print(np.sum(dataset_06h['agitation-four'] == 0),
      np.sum(dataset_06h['agitation-four'] == 1),
      np.sum(dataset_06h['agitation-four'] == 2),
      np.sum(dataset_06h['agitation-four'] == 3))

print(np.sum(dataset_06h['agitation-next'] == 0),
      np.sum(dataset_06h['agitation-next'] == 1))


1400 117 89 18
1752 135


In [36]:
columns_count = ['back-door', 'bathroom', 'bedroom', 'fridge-door',
                 'front-door', 'hallway', 'kitchen', 'lounge']

columns_contextual = ['total-events', 'unique-locations', 'active-location-ratio', 'private-to-public-ratio',
                      'location-entropy', 'location-dominance-ratio', 'back-and-forth-count', 'num-transitions']

columns_statistical = ['back-door-count-max', 'back-door-count-mean', 'back-door-count-std', 'back-door-count-sum',
                       'bathroom-count-max', 'bathroom-count-mean', 'bathroom-count-std', 'bathroom-count-sum',
                       'bedroom-count-max', 'bedroom-count-mean', 'bedroom-count-std', 'bedroom-count-sum',
                       'fridge-door-count-max', 'fridge-door-count-mean', 'fridge-door-count-std', 'fridge-door-count-sum',
                       'front-door-count-max', 'front-door-count-mean', 'front-door-count-std', 'front-door-count-sum',
                       'hallway-count-max', 'hallway-count-mean', 'hallway-count-std', 'hallway-count-sum',
                       'kitchen-count-max', 'kitchen-count-mean', 'kitchen-count-std','kitchen-count-sum',
                       'lounge-count-max', 'lounge-count-mean', 'lounge-count-std','lounge-count-sum']

columns_physiology = ['body-temperature', 'body-weight', 'diastolic-blood-pressure', 'heart-rate',
                      'muscle-mass', 'systolic-blood-pressure', 'total-body-water', 'skin-temperature']

columns_labels = ['blood-pressure', 'body-water', 'pulse', 'weight', 'body-temperature-label']

# feature_names = columns_count + columns_contextual + columns_statistical + columns_physiology + columns_labels
feature_names = columns_count + columns_contextual + columns_statistical + columns_physiology

print(len(columns_count),
      len(columns_contextual),
      len(columns_statistical),
      len(columns_physiology),
      len(columns_labels),
      len(feature_names))



8 8 32 8 5 56


In [ ]:
# Imputation

dataset = dataset_06h

print(dataset.shape)

dataset.drop(columns_labels, axis=1, inplace=True)

print(dataset.shape)


import utils_data
importlib.reload(utils_data)
from utils_data import hierarchical_imputation, hierarchical_imputation_columns_to_exclude

print(dataset.isna().sum().sum())

dataset = hierarchical_imputation_columns_to_exclude(dataset, columns_to_exclude=['id',
                                                                                  'date',
                                                                                  '6h',
                                                                                  'agitation',
                                                                                  'agitation-next',
                                                                                  'agitation-four'])
print(dataset.isna().sum().sum())
temp = dataset.drop([                                                             'id',
                                                                                  'date',
                                                                                  '6h',
                                                                                  'agitation',
                                                                                  'agitation-next',
                                                                                  'agitation-four'], axis=1, inplace=False)
print(temp.isna().sum().sum())


(10790, 67)
(10790, 62)
107888
17803
0


In [37]:
# Sequential

segment_length = 3

import utils_data
importlib.reload(utils_data)
from utils_data import segment_dataframe

dataset.fillna(0, inplace=True)

sements, labels = segment_dataframe(dataset, segment_length)

x = np.array(sements)
y = np.array(labels)

print(x.shape, y.shape)
display(x, y)

np.sum(y)


(10622, 3, 56) (10622,)


array([[[  0.    ,   0.    ,   0.    , ..., 136.4271,  49.8206,
          34.2843],
        [  0.    ,   0.    ,   0.    , ..., 136.4271,  49.8206,
          34.2843],
        [ 12.    ,   5.    ,  18.    , ..., 165.    ,  50.6   ,
          34.2843]],

       [[  0.    ,   0.    ,   0.    , ..., 136.4271,  49.8206,
          34.2843],
        [ 12.    ,   5.    ,  18.    , ..., 165.    ,  50.6   ,
          34.2843],
        [  2.    ,   2.    ,   6.    , ..., 136.4271,  49.8206,
          34.2843]],

       [[ 12.    ,   5.    ,  18.    , ..., 165.    ,  50.6   ,
          34.2843],
        [  2.    ,   2.    ,   6.    , ..., 136.4271,  49.8206,
          34.2843],
        [  0.    ,   0.    ,   3.    , ..., 130.7188,  51.76  ,
          34.56  ]],

       ...,

       [[  0.    ,  10.    ,  21.    , ..., 130.7188,  51.76  ,
          34.56  ],
        [  0.    ,  10.    ,  19.    , ..., 130.7188,  51.76  ,
          34.56  ],
        [  0.    ,  13.    ,  16.    , ..., 130.7188,  51

array([0, 0, 0, ..., 0, 0, 0], shape=(10622,))

np.int64(135)

In [38]:
# # Ablation
# dataset_CNT_STA_CTX_PHY_LBL = dataset
# dataset_CNT_STA_CTX_PHY_LBL.to_csv(os.path.join(root,   'dataset_06h_imputed_CNT_STA_CTX_PHY_LBL.csv'), index=False)
# print(dataset_CNT_STA_CTX_PHY_LBL.shape)

# dataset_CNT_STA_CTX_PHY = dataset.drop(columns_labels, axis=1, inplace=False)
# dataset_CNT_STA_CTX_PHY.to_csv(os.path.join(root,       'dataset_06h_imputed_CNT_STA_CTX_PHY.csv'), index=False)
# print(dataset_CNT_STA_CTX_PHY.shape)

# dataset_CNT_STA_CTX = dataset.drop(columns_physiology, axis=1, inplace=False)
# dataset_CNT_STA_CTX.to_csv(os.path.join(root,       'dataset_06h_imputed_CNT_STA_CTX.csv'), index=False)
# print(dataset_CNT_STA_CTX.shape)


# dataset_CNT_STA = dataset.drop(columns_contextual + columns_physiology, axis=1, inplace=False)
# dataset_CNT_STA.to_csv(os.path.join(root,           'dataset_06h_imputed_CNT_STA.csv'), index=False)
# print(dataset_CNT_STA.shape)

# dataset_CNT = dataset.drop(columns_statistical + columns_contextual + columns_physiology, axis=1, inplace=False)
# dataset_CNT.to_csv(os.path.join(root,           'dataset_06h_imputed_CNT.csv'), index=False)
# print(dataset_CNT.shape)

In [ ]:
# ROCKET
model_name = 'rocket'

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


# cv = LeavePGroupsOut(n_groups=1)
# for train_idx, test_idx in cv.split(x, y, groups=p):
#     participant = np.unique(p[test_idx])[0]


# cv = KFold(n_splits=x.shape[0], shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = np.unique(p[test_idx])[0]

cv = KFold(n_splits=5, shuffle=True, random_state=seed)
for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
    participant = fold

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])


    num_samples, time_steps, num_features = x_train.shape
    x_train_flat = x_train.reshape(-1, num_features)
    x_test_flat = x_test.reshape(-1, num_features)

    scaler = StandardScaler()
    scaler.fit(x_train_flat)
    x_train_scaled = scaler.transform(x_train_flat)
    x_test_scaled = scaler.transform(x_test_flat)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train_scaled)
    x_train_scaled = normalizer.transform(x_train_scaled)
    x_test_scaled = normalizer.transform(x_test_scaled)

    x_train = x_train_scaled.reshape(num_samples, time_steps, num_features)
    x_test = x_test_scaled.reshape(x_test.shape[0], time_steps, num_features)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    rocket = Rocket(num_kernels=1_000)
    rocket.fit(x_train)
    x_train_transformed = rocket.transform(x_train)
    x_test_transformed = rocket.transform(x_test)


    # ----- Gradient Boosting Classifier
    model = GradientBoostingClassifier(
        n_estimators=100,  # match LightGBM
        learning_rate=0.001,  # match LightGBM
        max_depth=4,  # similar to LightGBM default tree depth
        subsample=1.0,  # default
        random_state=seed
    )
    # model.fit(x_train, y_train)
    model.fit(x_train_transformed, y_train, sample_weight=compute_sample_weight(class_weight='balanced', y=y_train))
    y_probs = model.predict_proba(x_test_transformed)[:, 1]
    

    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, (y_probs >= .5).astype(int))


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

# Print results
print(f"AUC-ROC:    {auc_roc:.4f}")
print(f"AUC-PR:     {auc_pr:.4f}")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {pre:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(confusion_matrix(Y_TRUES, Y_PREDS))


metrics = {
    'auc-roc': f"{auc_roc:.4f}",
    'auc-pr': f"{auc_pr:.4f}",
    'accuracy': f"{acc:.4f}",
    'precision': f"{pre:.4f}",
    'recall': f"{rec:.4f}",
    'f1-score': f"{f1:.4f}",
    'sensitivity': f"{sensitivity:.4f}",
    'specificity': f"{specificity:.4f}",
    'tn': str(int(tn)),
    'fp': str(int(fp)),
    'fn': str(int(fn)),
    'tp': str(int(tp))
}

pd.DataFrame([metrics]).to_csv(
    model_name + '.csv', index=False)

1 8497 2125
